**The Thumbnail Task for Hayk**

### Import packages

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import requests
import time
import random
from PIL import Image, ImageDraw, ImageFilter

%load_ext autoreload
%autoreload 2

In [ ]:
# Run this
def crop_max_square(pil_img):
    return crop_center(pil_img, min(pil_img.size), min(pil_img.size))

In [ ]:
# Run this
def crop_center(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 2,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 2,
                         (img_height + crop_height) // 2))

### 1) read todo list

In [ ]:
media_df = pd.read_csv("../raw_data/url_df_redflag_selection.csv",index_col="gbifID")
media_df_nodupl = media_df[~media_df.index.duplicated(keep='first')]
path = '../raw_data/images_dist_list/'
file_todo = 'list_3.npy' # The list for Hayk
files_td = np.load(path+file_todo, allow_pickle=True)

### 2) scrap images - check if already there beforehand

In [ ]:
%%time
## foldername for the thumbnails
fold_n_thumb = '../raw_data/thumbnails/'
fi_n = os.listdir(fold_n_thumb)
#fi_n.remove('.DS_Store') ## in case you have a mac - this might be necessary
files = [os.path.splitext(filename)[0] for filename in os.listdir(fold_n_thumb)]
#files.remove('.DS_Store') ## in case you have a mac - this might be necessary

## read out the already existing files in the folder...
#media_df_nodupl = media_df[~media_df.index.duplicated(keep='first')]
#list_indexes = media_df_nodupl.index
#diff_li = list(set(list_indexes) - set(files))
diff_li = list(set(files_td) - set(files))
## shuffle the list - this might be necessary since requests at the same api in a row might
## denie access
random.shuffle(diff_li)

## request and store images in "images" folder
for i in range(5): # range(1000):
    try:
        r = requests.get(media_df_nodupl.loc[int(diff_li[i]),'identifier'], stream=True) #Get request on full_url
#         print(r) # for debugging
        with open(f"../raw_data/images/{int(diff_li[i])}.jpg", 'wb') as f: 
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f)
    except:
#         print("error") # for debugging
        continue
        
    time.sleep(random.uniform(0.1, 0.3))

### 3) crop and scale images

In [ ]:
## get images from images folde
fold_n_im = '../raw_data/images/'
fi_n = os.listdir(fold_n_im)
# fi_n.remove('.DS_Store') # Use on Mac

## crop and scale images to thumnails folder
for i in range(len(fi_n)):
    try:
        image = Image.open(f"../raw_data/images/{fi_n[i]}")
        image = crop_max_square(image).resize((300, 300))
        image.save(f"../raw_data/thumbnails/{fi_n[i]}") 
    except:
        continue